In [1]:
from salad.datasets import DigitsLoader
import torch
from salad.utils import evaluate

## Generate predictions for digit dataset multi-class training

In [2]:
import glob

def get_data(source):

    datasets = ['mnist', 'usps', 'synth', 'svhn']
    datasets.pop(datasets.index(source))
    datasets = [source] + datasets

    loader   = DigitsLoader('/tmp/data', datasets, download=True, shuffle=False, batch_size=64, drop_last=False,
                                                   normalize=True, num_workers=4, split="test")
    
    return loader, datasets

def run_eval(source, datasets, loader):

    fname = list(glob.glob("log/digits/{}/*_MultidomainBCESolver/*-checkpoint-ep99.pth".format(source)))
    #assert len(fname) == 1, fname
    fname = fname[0]
    
    print(fname)

    acc = {}

    for adapt in datasets:
        for i, target in enumerate(datasets):
            lbl, pred, _ = evaluate.evaluate([fname], loader.datasets[i], domain = datasets.index(adapt))
            acc[(source, adapt, target)] = [lbl, pred]
            
    return acc

In [3]:
predictions = {}
sources = ['mnist', 'usps', 'synth', 'svhn']

for source in sources:
    loader, datasets = get_data(source)
    predictions.update(run_eval(source, datasets, loader))

Normalize data
Normalize data
Extracting /tmp/data/zip.train.gz
Normalize data
Extracting /tmp/data/synth_test_32x32.mat?raw=true
Normalize data
Using downloaded and verified file: /tmp/data/test_32x32.mat
log/digits/mnist/20180926-125754_MultidomainBCESolver/20180926-125754-checkpoint-ep99.pth


100%|██████████| 407/407 [00:04<00:00, 88.40it/s]


Normalize data
Extracting /tmp/data/zip.train.gz
Normalize data
Normalize data
Extracting /tmp/data/synth_test_32x32.mat?raw=true
Normalize data
Using downloaded and verified file: /tmp/data/test_32x32.mat


  0%|          | 0/114 [00:00<?, ?it/s]

log/digits/usps/20180926-130054_MultidomainBCESolver/20180926-130054-checkpoint-ep99.pth


...

100%|██████████| 407/407 [00:04<00:00, 85.92it/s]


Normalize data
Using downloaded and verified file: /tmp/data/test_32x32.mat
Normalize data
Normalize data
Extracting /tmp/data/zip.train.gz
Normalize data
Extracting /tmp/data/synth_test_32x32.mat?raw=true


  0%|          | 0/407 [00:00<?, ?it/s]

log/digits/svhn/20180926-143322_MultidomainBCESolver/20180926-143322-checkpoint-ep99.pth


100%|██████████| 150/150 [00:01<00:00, 79.83it/s]


In [9]:
import h5py

with h5py.File("results/digit_predictions.hdf5") as ds:
    
    for key in predictions:
        
        ds_key = "-".join(key)
        
        grp = ds.require_group(ds_key)
        
        lbls, pred = predictions[key]
        
        grp["prediction"] = pred
        grp["labels"] = lbls
    

In [10]:
! du -h results/digit_predictions.hdf5

39M	results/digit_predictions.hdf5
